In [1]:
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing 

In [3]:
raw_data = pd.read_csv('../data/audio_data_genre.csv').drop(columns=['filename'])

In [ ]:
print(raw_data['label'].unique())

In [ ]:
COLS_USED = raw_data.columns.to_list()
COLS_TRAIN = [col for col in COLS_USED if col != 'label']

In [ ]:
data = {
    'label': ['blues', 'classical','country','disco','hiphop','jazz','metal','pop',
 'reggae','rock']
}
raw_data = pd.DataFrame(data)

print("Before transformation:")
print(raw_data.head())

label_encoder = preprocessing.LabelEncoder()
raw_data['label_encoded'] = label_encoder.fit_transform(raw_data['label'])

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("\nAfter transformation:")
print(raw_data.head())

print("\nLabel mapping:")
for original_label, encoded_label in label_mapping.items():
    print(f"{encoded_label}: {original_label}")

In [ ]:
print(raw_data['label'].unique())

In [ ]:
raw_data = raw_data[COLS_USED]

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    raw_data.drop("label", axis=1),
    raw_data["label"],
    test_size=0.20,
    random_state=42,
)

df_x_test, df_y_test = df_x_test.reset_index(drop=True), \
  df_y_test.reset_index(drop=True)
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

In [ ]:
model = Sequential()

model.add(Dense(50, input_dim=x_train.shape[1], activation='relu')) 
model.add(Dense(25, activation='relu')) 
model.add(Dense(12, activation='relu')) 
model.add(Dense(1)) 
model.compile(loss='mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=5, verbose=1, mode='auto',
        restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
        callbacks=[monitor], verbose=2,epochs=1000)

In [8]:
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred,y_test))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 6.008439590991187


In [ ]:
from tabgan.sampler import GANGenerator

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999,
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state":
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False
          ).generate_data_pipe(df_x_train, df_y_train,
          df_x_test, deep_copy=True, only_adversarial=False, 
          use_adversarial=True)

In [ ]:
pred = model.predict(gen_x.values)
score = np.sqrt(metrics.mean_squared_error(pred,gen_y.values))
print("Final score (RMSE): {}".format(score))

In [ ]:
data_augmented = pd.concat([raw_data, gen_x])

In [ ]:
data_augmented.shape

In [ ]:
data_augmented.to_csv('./data/audio_data_genre_augmented.csv', index=False)